In [46]:
from collections import Counter
import pandas as pd
import math
import pysam
import operator
import re

class PileupRecord:
    
    def __init__(self,line):
        fields = line.split("	")
        self.seq = fields[0]
        self.pos = int(fields[1])
        self.ref = fields[2]
        self.rCount = int(fields[3])
        self.rRes = fields[4]
        self.qual = fields[5][:-1]
        
    def print(self):
        print('{}    {}    {}    {}    {}    {}'.format(self.seq, self.pos, self.ref, self.rCount, self.rRes, self.qual))


def VCFHeader():
    VCFheader = pysam.VariantHeader()
    VCFheader.add_sample('sample1')
    VCFheader.add_line("##ALT=<ID=*,Description=Represents allele(s) other than observed.>")
    VCFheader.add_line("##FORMAT=<ID=GT,Number=1,Type=String,Description=Genotype>")
    VCFheader.add_line("##FORMAT=<ID=VAF,Number=1,Type=String,Description=Called genome probability>")
    
def VCFBodyLine(rec, alts, genotype, p, VCF_out):
        record = VCF_out.header.new_record()
        record.contig = rec.seq
        record.ref = rec.ref
        record.alts = alts
        record.pos = rec.pos
        record.samples['HCC1143BL']['GT'] = genotype
        record.samples['HCC1143BL']['VAF'] = str(p)
    
        return record
    
def DetermineIndelString(readResult):
    
    ind_num = [ind.start() for ind in (re.finditer(r'[0-9]', readResult))]
    if len(ind_num) > 0:
        string = readResult[2:ind_num[0]]
        if len(ind_num) != 1:
            for i in range(0,len(ind_num)-1):
                string += int(readResult[ind_num[i]])*readResult[ind_num[i]+1:ind_num[i+1]]
        string += int(readResult[ind_num[-1]])*readResult[ind_num[-1]+1:]
    else:
        string = readResult[2:]
    return string

def DetectPolymorphicSite(readResult):
    
    '''Detects variants at particular location, returns counts and types of them.
       input: string, info about particular location alignment results from all reads
       output: dataframe, columns: variant, count, type'''
    
    readResult = readResult.upper().replace(',','.')

    irrelevant = list(set(re.findall(r'\^[\W][^\.]', readResult)))
    for s in irrelevant:
        readResult = readResult.replace(s,'')
    
    occ = re.findall(r'[\.][+-][ACGT]*[0-9]*[ACGT]*[0-9]*[ACGT]*', readResult)
    var = []
    variants = list(set(occ))
    variants1 = [DetermineIndelString(variant) for variant in variants]
    varCounts = [occ.count(indel) for indel in variants]
    varTypes = ['indel']*len(variants1)
    for i in range(0,len(variants1)):
        var.append([variants1[i], varCounts[i], varTypes[i]])
        
    for s in variants:
        readResult = readResult.replace(s,'')
    
    occ = re.findall(r'[AGCT]', readResult)
    SNVs = list(set(occ))
    SNVCounts = [occ.count(SNV) for SNV in SNVs]
    SNVTypes = ['SNV']*len(SNVs)
    for i in range(0,len(SNVs)):
        var.append([SNVs[i], SNVCounts[i], SNVTypes[i]])

    matchCount = len(re.findall(r'[\.]', readResult))
    var.append(['.', matchCount, 'match'])
    
    var.sort(key = lambda x: x[1], reverse = True)

    if len(var) > 2:
        return var[:2]
    else:
        return var 


def Genotyping(var,Qual):
    
    '''Determines genotype.'''
    
    if len(var) == 1:
        if var[0][2] == 'match':
            genotype = (0,0)
        else:
            genotype = (1,1)
        P = []
    if len(var) == 2:
        pr = [0]*3
        k1 = var[0][1]
        k2 = var[1][1]
        # k1 = 40
        # k2 = 1
        if var[0][2] == 'match' or var[0][2] == 'SNV':
            p0 = 0.8
        else:
            p0 = 0.6
            
        if var[1][2] == 'match' or var[1][2] == 'SNV':
            p1 = 0.8
        else:
            p1 = 0.6
        p2 = (p0 + p1)/2

        pr[0] = math.factorial(k1+k2)//math.factorial(k1)//math.factorial(k2)*(p0**k1)*(1-p0)**(k2) # a1a1
        pr[1] = math.factorial(k1+k2)//math.factorial(k1+k2)//math.factorial(0)*(p2**(k1+k2))*(1-p2)**0 # a1a2
        pr[2] = math.factorial(k1+k2)//math.factorial(k2)//math.factorial(k1)*p1**k2*(1-p1)**(k1) # a2a2
    
        P = [pr[0]/(pr[0]+pr[1]+pr[2]), pr[1]/(pr[0]+pr[1]+pr[2]), pr[2]/(pr[0]+pr[1]+pr[2])]
        index, value = max(enumerate(P), key=operator.itemgetter(1))
    
        if var[0][2] == 'match': 
            if index == 0:
                genotype = (0,0)
            elif index == 1:
                genotype = (0,1)
            else:
                genotype =(1,1)
        elif var[1][2] == 'match':
            if index == 0:
                genotype = (1,1)
            elif index == 1:
                genotype = (0,1)
            else:
                genotype = (0,0)
            P.reverse()
        else:
            if index == 0:
                genotype = (1,1)
            elif index == 1:
                genotype = (1,2)
            else:
                genotype = (2,2)
  
    return genotype, P

def DetermineAltsField(polymorphic_site):
    
    polymorphic_site = [elem for elem in polymorphic_site if(elem[0] != '.')]
    if len(polymorphic_site) == 0:
        alts = ['.']
    else:
        alts = [p[0] for p in polymorphic_site]
    
    return alts


def AvgQual(qual):
    qual_prob = []
    for char in qual:
        qual_prob.append(1 - 10**-((ord(char)-33)/10.0))
    
    return sum(qual_prob)/len(qual_prob)

In [48]:
pileup_file = open("merged-normal.pileup")
VCF_read = pysam.VariantFile("merged-normal.mpileup.called.vcf", "r")
VCFheader = VCF_read.header
VCFheader.add_line("##FORMAT=<ID=VAF,Number=1,Type=String,Description=Called genome probability>")
VCF_out = pysam.VariantFile("VCFout.vcf", "w", header = VCFheader)

for line in pileup_file:
    rec = PileupRecord(line)
    
    polymorphic_site = DetectPolymorphicSite(rec.rRes,rec.rCount)
    Qual = AvgQual(rec.qual)
    genotype, probs = Genotyping(polymorphic_site, Qual)
    
    if genotype != (0,0):
        alts = DetermineAltsField(polymorphic_site)
        
        VCF_line = VCFBodyLine(rec, alts, genotype, round(max(probs),4), VCF_out)
        VCF_out.write(VCF_line)
        
VCF_out.close()